# Tutorial: Variational algorithms on Hybrid jobs

## Hybrid Jobs

Amazon Braket Hybrid Jobs offers a way for you to run hybrid quantum-classical algorithms that require both classical resources and quantum processing units (QPUs). Hybrid Jobs is designed to spin up the requested classical compute, run your algorithm, and release the instances after completion so you only pay for what you use. This workflow is ideal for long-running iterative algorithms involving both classical and quantum resources. In this notebook we show you how to run such a job via the qiskit-braket provider.

Read more at https://docs.aws.amazon.com/braket/latest/developerguide/braket-jobs.html

## Prepare Files

### Prepare Hybrid Job Script

The job script defines the specific quantum logic to be executed on the quantum processor and the classical processing that the output is saved to, in this case, setting up the device backend and saving the job results to the EC2 instance.


In [1]:
! cat ./data/2_hybrid_jobs/job_script.py

"""Example of Hybrid Job payload with VQE."""
from braket.jobs import save_job_result
from qiskit.quantum_info import SparsePauliOp
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import BackendEstimator

from qiskit_braket_provider import AWSBraketProvider


def main():
    # Sets up the "SV1" backend the quantum device, a default simulator provided by Amazon Braket.
    backend = AWSBraketProvider().get_backend("SV1")

    h2_op = SparsePauliOp(
        ["II", "IZ", "ZI", "ZZ", "XX"],
        coeffs=[
            -1.052373245772859,
            0.39793742484318045,
            -0.39793742484318045,
            -0.01128010425623538,
            0.18093119978423156,
        ],
    )

    estimator = BackendEstimator(
        backend=backend,
        options={"seed_simulator": 42, "seed_transpiler": 42, "shots": 10},
        skip_transpilation=False,
    )
    ansatz 

In summary, this script sets up the hybrid job script that executes a VQE computation using a the Amazon Braket SV1 default simulator as the quantum backend and then saves the results of the computation to the EC2 instance the hybrid job is running on.

## Running your job
Amazon Braket provides a container for Hyrbid Jobs that has the qiskit-braket container and associated dependencies installed. To use the base container, call `retrieve_image(Framework.BASE, AwsSession().region)` as demonstrated below:

In [5]:
from pprint import pprint
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.image_uris import Framework, retrieve_image

job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="data/2_hybrid_jobs/job_script.py",
    entry_point="job_script:main",
    wait_until_complete=False,
    job_name="qiskit-braket-vqe-6",
    image_uri=retrieve_image(Framework.BASE, AwsSession().region),
)


In [7]:
result = job.result()
pprint(result)

{}


In [ ]:
import matplotlib.pyplot as plt

# Sample data
result = {
    'VQE': {
        'eigenstate': {'01': 0.8366600265340756, '11': 0.5477225575051661},
        'eigenvalue': -1.6823050629682066,
        'optimal_parameters': [2.965910707685307, -4.658133055892289, -4.3956015975227265, -3.682189424847994, 0.3499389384822065, -2.55160387237748, 2.0897918250881915, 1.0061448749802206],
        'optimal_point': [2.965910707685307, -4.658133055892289, -4.3956015975227265, -3.682189424847994, 0.3499389384822065, -2.55160387237748, 2.0897918250881915, 1.0061448749802206],
        'optimal_value': -1.6823050629682066
    }
}

# Extract data for visualization
eigenstate = result['VQE']['eigenstate']
optimal_parameters = result['VQE']['optimal_parameters']
optimal_point = result['VQE']['optimal_point']

# Create a 3x1 grid of subplots
plt.figure(figsize=(8, 12))

# Plot Eigenstate Probabilities
plt.subplot(3, 1, 1)
plt.bar(eigenstate.keys(), eigenstate.values())
plt.xlabel('Eigenstate')
plt.ylabel('Probability')
plt.title('Eigenstate Probabilities')

# Plot Optimal Parameters
plt.subplot(3, 1, 2)
plt.plot(optimal_parameters, marker='o', linestyle='--', color='b', label='Optimal Parameters')
plt.xlabel('Parameter Index')
plt.ylabel('Parameter Value')
plt.title('Optimal Parameters')

# Plot Optimal Point Values
plt.subplot(3, 1, 3)
plt.plot(optimal_point, marker='o', linestyle='--', color='g', label='Optimal Point')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Optimal Point Values')

# Adjust spacing between subplots
plt.subplots_adjust(hspace=0.5)

# Show the plot
plt.tight_layout()
plt.show()
